___Горнатенко Даниил, ПИ22-5___

## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [10]:
import random
N = 1000000
A = [random.randint(0, 1000) for i in range(N)]
B = [a + 100 for a in A]
sum(B) / N

599.938381

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [12]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.rand(2000000, 4), columns=['1', '2', '3', '4'])

df['key'] = [chr(np.random.randint(97, 122)) for i in range(2000000)]

df[df['key'].str.contains('^[a-eA-E]')]

,1,2,3,4,key
2,0.869806,0.153813,0.405403,0.468079,b
17,0.531765,0.603818,0.024534,0.032101,a
20,0.843399,0.905415,0.429450,0.777628,b
25,0.927212,0.687816,0.324023,0.302171,d
26,0.802025,0.415647,0.772678,0.000638,c
...,...,...,...,...,...
1999965,0.939814,0.292776,0.584670,0.720668,e
1999979,0.545696,0.556813,0.194879,0.036872,e
1999993,0.997801,0.534124,0.619864,0.622083,e
1999994,0.933772,0.628494,0.645236,0.889022,d


## Лабораторная работа 3

In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [13]:
import pandas as pd
import time

recipes = pd.read_csv('recipes_sample.csv', index_col=0, delimiter=',')
reviews = pd.read_csv('reviews_sample.csv', index_col=0, delimiter=',')

reviews['date'] = pd.to_datetime(reviews['date'])

def mean1(reviews):
    sumall = 0
    counter = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            sumall += row['rating']
            counter += 1
    return sumall / counter

def mean2(reviews):
    sumall = 0
    counter = 0
    for index, row in reviews.loc[reviews['date'].dt.year == 2010].iterrows():
        sumall += row['rating']
        counter += 1
    return sumall / counter

def mean3(reviews):
    filtered_reviews = reviews.loc[reviews['date'].astype(str).str.contains('2010')]
    return filtered_reviews['rating'].mean()

print(f'1) {mean1(reviews)}')
print(f'2) {mean2(reviews)}')
print(f'3) {mean3(reviews)}')

print('\n')

start_time = time.time()
res1 = mean1(reviews)
end_time = time.time()
print(f'mean1: {res1}, time - {end_time - start_time}')

start_time = time.time()
res2 = mean2(reviews)
end_time = time.time()
print(f'mean2: {res2}, time - {end_time - start_time}')

start_time = time.time()
res3 = mean3(reviews)
end_time = time.time()
print(f'mean3: {res3}, time - {end_time - start_time}')

1) 4.4544402182900615
2) 4.4544402182900615
3) 4.4544402182900615


mean1: 4.4544402182900615, time - 1.8806769847869873
mean2: 4.4544402182900615, time - 0.1675550937652588
mean3: 4.4544402182900615, time - 0.31706786155700684


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [14]:
print('Вариант A, так как мы итерируемся абсолютно по всем строкам, а в других вариантах мы проходимся\
только по отфильтрованным (только отзывы за 2010 год) строкам, что значительно уменьшает количество обрабатываемых\
данных, вследствие чего сильно увеличивает время выполнения функции')

Вариант A, так как мы итерируемся абсолютно по всем строкам, а в других вариантах мы проходимсятолько по отфильтрованным (только отзывы за 2010 год) строкам, что значительно уменьшает количество обрабатываемыхданных, вследствие чего сильно увеличивает время выполнения функции


In [15]:
!pip install line_profiler
%load_ext line_profiler

%lprun -f mean1 mean1(reviews)

     |████████████████████████████████| 151 kB 766 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for line-profiler: filename=line_profiler-4.0.3-cp39-cp39-macosx_11_0_arm64.whl size=85419 sha256=2dd0a0a3eb3745718c18ecd299556cc06bdd6ec172f79af0e79f9d96178b61d8
  Stored in directory: /Users/daniilgornatenko/Library/Caches/pip/wheels/5e/f9/e4/51d4cec1d9283c683a7edea0aad7c5bf1bad70c1d886de3cb1
Successfully built line-profiler


In [16]:
def mean1_2(reviews):
    filtered_reviews = reviews[reviews['date'].dt.year == 2010]
    count = len(filtered_reviews)
    if count > 0:
        return filtered_reviews['rating'].apply(lambda x: float(x)).sum() / count
    else:
        return None

%lprun -f mean1_2 mean1_2(reviews)

print('Таким образом, используя лямбда-функцию и метод apply, мы сокращаем время выполнения программы, так как\
в памяти устройства во время выполнения функции не сохраняются данные, а в предыдущей версии функции сохранялись')

Таким образом, используя лямбда-функцию и метод apply, мы сокращаем время выполнения программы, так какв памяти устройства во время выполнения функции не сохраняются данные, а в предыдущей версии функции сохранялись


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

%lprun -f get_word_reviews_count get_word_reviews_count(reviews) # Total time: 51.2559 s

from collections import defaultdict

def get_word_reviews_count2(df):
    word_reviews = defaultdict(list) # для хранения списка рецептов, в которых встречается каждое слово
    word_reviews_count = defaultdict(int) # для хранения количества отзывов с каждым словом
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        for word in review.split():
            word_reviews[word].append(recipe_id)
    for word in word_reviews:
        word_reviews_count[word] = len(word_reviews[word])
# +заменили сплит на более быстрый вариант работы со строкой как со списком символов, сократили время выполнения функции в 2 раза: Total time: 26.2478 s
 
%lprun -f get_word_reviews_count2 get_word_reviews_count2(reviews)